Step 1) Ask a question: Classify whether or not an individual survives the Titanic ship catastrophe based on indivdual and ship characteristics. Will this indivdual survive the Titanic catastrophe?
 
Step 2) Essentially we want to take passanger information and predict if this passanger will survive the Titanic. 

Step 3) This is a classification question as we are predicitng a label for survival, either they survived or they died. 
    In our DataFrames X is Features or sample data and y are our labels or targets


In [62]:
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import(
    ensemble, 
    preprocessing, 
    tree, 
)
from sklearn.metrics import (
    auc, 
    confusion_matrix, 
    roc_auc_score, 
    roc_curve,
)
from sklearn.model_selection import (
    train_test_split, 
    StratifiedKFold, 
)
from yellowbrick.classifier import(
        ConfusionMatrix,
        ROCAUC,
)
from yellowbrick.model_selection import (
    LearningCurve, 
) 


In [3]:
df = pd.read_csv(r"C:\Users\alexn\Documents\MachineLearning\titanic\train.csv")
df.head()
print(df.shape)

(891, 12)


In [5]:
df.describe().iloc[:,:2]

,PassengerId,Survived
count,891.000000,891.000000
mean,446.000000,0.383838
std,257.353842,0.486592
min,1.000000,0.000000
25%,223.500000,0.000000
50%,446.000000,0.000000
75%,668.500000,1.000000
max,891.000000,1.000000


In [6]:
df.isnull().mean() 
# We will impute or derive the data for missing values in the age column 

PassengerId    0.000000
Survived       0.000000
Pclass         0.000000
Name           0.000000
Sex            0.000000
Age            0.198653
SibSp          0.000000
Parch          0.000000
Ticket         0.000000
Fare           0.000000
Cabin          0.771044
Embarked       0.002245
dtype: float64

In [7]:
mask = df.isnull().any(axis=1)
mask.head()

0     True
1    False
2     True
3    False
4     True
dtype: bool

In [19]:
df.columns
df.Sex.value_counts(dropna=False) # Warning: Pandas usually drops null values, if you want to include them add dropna=False

Sex
male      577
female    314
Name: count, dtype: int64

In [22]:
df.columns
df.Embarked.value_counts(dropna=False)

Embarked
S      644
C      168
Q       77
NaN      2
Name: count, dtype: int64

Create Features: We drop columns that have no variance or signal. Irreleavant features like "is human" that had 1 for every sample would not be providing a lot of information 

In [25]:
# The pandas .drop method can drop either rows or columns 

name = df.Name
name.head(3)


0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
Name: Name, dtype: object

In [26]:
df.columns


Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [33]:
df = df.drop('Cabin', axis=1)

In [37]:
# We need to create dummy columns from string columns. Pandas has a convenient get_dummies function 

In [47]:
df = pd.get_dummies(df, drop_first=True)
df.columns


Index(['Embarked_C', 'Embarked_Q', 'Embarked_S', 'Fare', 'Parch',
       'PassengerId', 'Pclass', 'Sex_female', 'SibSp', 'Survived'],
      dtype='object')

 Create a DataFrame with the features (X) and a series (y)

In [59]:
y = df.Survived # series 
X = df.drop(columns='Survived') # DataFrame 

In [60]:
# Alternatively, use pyjanitor
import janitor as jn 

X, y = jn.ml.get_features_targets(df, target_columns='Survived')

C:\Users\alexn\anaconda3\Lib\site-packages\janitor\utils.py:365: DeprecationWarning: target_columns is deprecated; use target_column_names
  warn(


Sample Data: We always want to train and test on different data. Otherwise we won't know how well the model generalizes to data is hasn't seen before. We will use scikit-learn to pull out 30% for testing

In [71]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

Impute Data: The age column has missing values, we need to impute age from the numeric values. 